In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

Set up Python environment

***fine-tune LLaMA 2 models on  datasets***



In [2]:
import argparse
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [4]:
import torch
torch.cuda.is_available()

True

In [31]:
import pandas as pd
import time

In [32]:
file_path = "/home/nasrallah_hassan/AmharicAI-AdGen/data/parsed/all_data.csv"
dataset_file_path = "/content/drive/My Drive/Datasets/W_7/ad-not-add.csv"

In [33]:
df = pd.read_csv(file_path)

In [34]:
df.tail()

,text,label
4701,የኑዌር ዞን በርካታ ደጋፊዎች ጅማ እስታዲየም በመገኘት ጨዋታውን እያደመቁ...,Not ad
4702,🇪🇹 #የኢትዮጵያ_አንደኛ_ሊግ_ምድብ_ለ_ሶዶ ⌚️85' ✅ #ካማሺ_ከተማ...,Not ad
4703,ጎልልልልልልልል ነጌሌ ቦረና 77'⚽️አዲሱ ተሰማ,Not ad
4704,ጎልልልልልልልልልልል ካማሺ ከተማ 90+1',Not ad
4705,🇪🇹 #የኢትዮጵያ_አንደኛ_ሊግ_ምድብ_ሀ ⌚️85' ✅ #ነጌሌ_ቦረና 1-...,Not ad


In [35]:
dataset = df[['text','label']]
dataset.head()

,text,label
0,Tecno Spark 20 pro + \n\nአስገራሚ የቴክኖሎጂ ውጤቶችን ያካ...,Ad
1,#DELL PRECISION \n#EUROPE STANDARD \n🅑︎🅡︎🅐︎🅝︎🅓...,Ad
2,አይብ በሚዘጋጅበት ጊዜ ከቅቤው ከሚለየው ወተት ውስጥ ካለው የውሃ ክፍል ...,Ad
3,👨🏾‍⚕️🐠የአሳ ዘይት(ኦሜጋ -3 ፋቲ አሲድ)ለሰዉነታችን አስፈላጊ ከሚባሉ...,Ad
4,ቴሌግራም👉 👨🏾‍⚕️የአሳ ዘይት(ኦሜጋ -3 ፋቲ አሲድ)ለሰዉነታችን አስፈላ...,Ad


In [36]:
dataset.shape

(4706, 2)

In [37]:
dataset

,text,label
0,Tecno Spark 20 pro + \n\nአስገራሚ የቴክኖሎጂ ውጤቶችን ያካ...,Ad
1,#DELL PRECISION \n#EUROPE STANDARD \n🅑︎🅡︎🅐︎🅝︎🅓...,Ad
2,አይብ በሚዘጋጅበት ጊዜ ከቅቤው ከሚለየው ወተት ውስጥ ካለው የውሃ ክፍል ...,Ad
3,👨🏾‍⚕️🐠የአሳ ዘይት(ኦሜጋ -3 ፋቲ አሲድ)ለሰዉነታችን አስፈላጊ ከሚባሉ...,Ad
4,ቴሌግራም👉 👨🏾‍⚕️የአሳ ዘይት(ኦሜጋ -3 ፋቲ አሲድ)ለሰዉነታችን አስፈላ...,Ad
...,...,...
4701,የኑዌር ዞን በርካታ ደጋፊዎች ጅማ እስታዲየም በመገኘት ጨዋታውን እያደመቁ...,Not ad
4702,🇪🇹 #የኢትዮጵያ_አንደኛ_ሊግ_ምድብ_ለ_ሶዶ ⌚️85' ✅ #ካማሺ_ከተማ...,Not ad
4703,ጎልልልልልልልል ነጌሌ ቦረና 77'⚽️አዲሱ ተሰማ,Not ad
4704,ጎልልልልልልልልልልል ካማሺ ከተማ 90+1',Not ad


In [38]:
from datasets import Dataset
dataset['text'] = dataset['text'].astype(str)
data_dict = {"text": dataset['text'].tolist(), "label": dataset['label'].tolist()}
dataset = Dataset.from_dict(data_dict)

In [39]:
import random
from math import ceil
total_indices = len(dataset)

random.seed(150)
train_size = ceil(0.8 * total_indices)
train_indices = random.sample(range(total_indices), train_size)

In [40]:
test_indices= [i for i in range(total_indices) if i not in train_indices]

In [41]:
train_dataset = dataset.select(train_indices)
test_dataset = dataset.select(test_indices)
dataset = train_dataset
dataset_subset = test_dataset

In [42]:
from collections import Counter
Counter(dataset['label'])

Counter({'Ad': 1888, 'Not ad': 1876, 'label': 1})

In [43]:
print(dataset['text'][0])

ባንካችን ተደራሽነቱን ይበልጥ በማስፋፋት ከታች በሰንጠረዡ ከ274 እስከ 276ኛ የተገለፁትን ቅርንጫፎች ነሀሴ 13 ቀን 2015 ኣም በመክፈት አገልግሎት መስጠት መጀመሩን በደስታ እንገልፃለን፡፡ አማራ ባንክ ከባንክ ባሻገር! የአማራ ባንክ ትክክለኛ የማህበራዊ ትስስር ገፆች : : : : : : :


In [44]:
print(dataset_subset['text'][2])

ቴሌግራም👉 🏋️‍♀️()ብራንችድ ቼን አሚኖ አሲድ የሶስት አይነት ኤሰንሽያል አሚኖ አሲዶች :- ሉሲን፣ አይሶሶሉሲን እና ቫሊን ዉቅር ሲሆን፡። 👩🏽‍⚕️እነዚህ ኤሰንሽያል አሚኖ አሲዶች ሰውነት ዉስጥ ሊመረቱ የማይችሉ እና ከምግብ ወይም ከ ሰፕሊመንቶች መገኘት ያአለባቸው የአሚኖ አሲድ አይነቶች ናቸዉ። 👉ከእንቅስቃሴ ጋር ሲወሰድ ደግሞ የጡንቻ መጠን ሳይቀንስ የሰዉነት ስብን ለመቀነስ() ለማድረግ 👉 ጡንቻን ለመገንባት 👉በተጨማሪም የጡንቻን ጉዳት ለመከላከል ወይም ለመቀነስ 👉 የጉበት በሽታ ምልክቶችን ለማሻሻል በሆስፒታል ውስጥ በተሳካ ሁኔታ ጥቅም ላይ በመዋል ላይ ይገኛል። 📍ባህርዳር - ዋርካው (ምስራቅ ፀሀይ)የገበያ ማእከል ዋናው ገበያ ፒያሳ ህንፃ ጎን 1ኛ ፎቅ 83 ቁጥር 📍 አዳማ - ፖስታ ቤት ሶሬቲ ህንፃ ፊትለፊት ጀርመን ሲቲ ሞል 1ኛ ፎቅ ሱቅ ቁጥር 110 ⏱መደብራችን ከሰኞ - ቅዳሜ ከጠዋት 2 ሰኣት እስከ ምሽት 1ሰኣት, እሁድ ከጠዋቱ 3 ሰኣት እስከ ምሽቱ 11 ሰኣት ክፍት ነው። 📫ከቅርንጫፎቻችን ውጪ በሆኑ በየትኛውም የሀገሪቱ ክፍል ለምትገኙ በፓስታ ቤት በፍጥነት እንልካለን በመደወል ማዘዝ ይችላሉ ☎️9369 ላይ አሁኑኑ ይደውሉ ☎️


Function  to download LLaMA 2 model and its tokenizer. It requires a bitsandbytes configuration

In [45]:
from transformers import PreTrainedTokenizerFast
def load_model(model_name, bnb_config, ):
    n_gpus = torch.cuda.device_count()
    max_memory = f'{23000}MB'

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto", 
        max_memory = {i: max_memory for i in range(n_gpus)},
        trust_remote_code=True
    )

    # tokenizer = spm.SentencePieceProcessor(model_file=str(tokenizer_model))
    # tokenizer = SentencePieceBPETokenizer(vocab_file='token.vocab')
    tokenizer = AutoTokenizer.from_pretrained("misge10/amharic-tokenizer", use_auth_token=True)
    # tokenizer = PreTrainedTokenizerFast.from_pretrained("misge10/am-token")
    # tokenizer = PreTrainedTokenizerFast.from_pretrained("tokenizer")
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer


Pre-processing dataset




In [46]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('text', 'label',)
    Then concatenate them using two newline characters
    :param sample: Sample dictionary
    :return: Modified sample dictionary with the formatted prompt
    """

    INTRO_BLURB = "You are tasked with classifying messages as either advertisements (Ad) or non-advertisements (Not Ad) based on a comprehensive analysis of various factors."

    INSTRUCTION_KEY = "### Text:"
    RESPONSE_KEY = "Classification:"
    END_KEY = "### End"

    blurb = f"{INTRO_BLURB}\n\n"
    text = f"{INSTRUCTION_KEY}\n{sample['text']}\n\n"
    response = f"{RESPONSE_KEY}\n{sample['label']}\n\n"
    end = f"{END_KEY}"

    formatted_prompt = blurb + text + response + end

    sample["text"] = formatted_prompt

    return sample


In [47]:
dataset[0]

{'text': 'ባንካችን ተደራሽነቱን ይበልጥ በማስፋፋት ከታች በሰንጠረዡ ከ274 እስከ 276ኛ የተገለፁትን ቅርንጫፎች ነሀሴ 13 ቀን 2015 ኣም በመክፈት አገልግሎት መስጠት መጀመሩን በደስታ እንገልፃለን፡፡ አማራ ባንክ ከባንክ ባሻገር! የአማራ ባንክ ትክክለኛ የማህበራዊ ትስስር ገፆች : : : : : : :',
 'label': 'Not ad'}

In [48]:
create_prompt_formats(dataset[0])

{'text': 'You are tasked with classifying messages as either advertisements (Ad) or non-advertisements (Not Ad) based on a comprehensive analysis of various factors.\n\n### Text:\nባንካችን ተደራሽነቱን ይበልጥ በማስፋፋት ከታች በሰንጠረዡ ከ274 እስከ 276ኛ የተገለፁትን ቅርንጫፎች ነሀሴ 13 ቀን 2015 ኣም በመክፈት አገልግሎት መስጠት መጀመሩን በደስታ እንገልፃለን፡፡ አማራ ባንክ ከባንክ ባሻገር! የአማራ ባንክ ትክክለኛ የማህበራዊ ትስስር ገፆች : : : : : : :\n\nClassification:\nNot ad\n\n### End',
 'label': 'Not ad'}

In [49]:
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )


def preprocess_dataset(tokenizer, max_length: int, seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """

    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)

    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=["text", "label"],
    )

    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    dataset = dataset.shuffle(seed=seed)

    return dataset

**Create a bitsandbytes configuration**



In [50]:
''' This function, create_bnb_config(), is designed to create and return a
configuration object for quantization using the Bits and Bytes (BNB)
quantization scheme. '''
def create_bnb_config():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    return bnb_config

** LoRa configuration**



In [51]:
def create_peft_config(modules):
    """
    Create Parameter-Efficient Fine-Tuning config for the model
    :param modules: Names of the modules to apply Lora to
    """
    config = LoraConfig(
        r=16,  
        lora_alpha=64,  
        target_modules=modules,
        lora_dropout=0.1, 
        bias="none",
        task_type="CAUSAL_LM",
        # layers_to_transform=
    )

    return config

In [52]:
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit 
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names: 
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [53]:
def print_trainable_parameters(model, use_4bit=False):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel

        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    if use_4bit:
        trainable_params /= 2
    print(
        f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}"
    )


In [54]:
!pip install python-dotenv sentencepiece

**Train**

Now, we can pre-process our dataset and load our model using the set configurations


In [55]:
from dotenv import load_dotenv
from huggingface_hub import login

load_dotenv()
token = os.environ["huggingface_token"]
login(token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/nasrallah_hassan/.cache/huggingface/token
Login successful


In [56]:
!torch.cuda.empty_cache()

/bin/bash: -c: line 1: syntax error: unexpected end of file


In [57]:
model_name = "stabilityai/stablelm-zephyr-3b"
bnb_config = create_bnb_config()

model, tokenizer2 = load_model(model_name, bnb_config)

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.
/opt/miniconda/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:712: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [58]:
# tokenizer = Tokenizer.from_pretrained("dagim/amharic_tokenizer") 

In [63]:
import random

seed = 42
random.seed(50)

In [64]:
max_length = get_max_length(model)

dataset = preprocess_dataset(tokenizer2, max_length, seed, dataset)

Found max lenth: 4096
Preprocessing dataset...


Map:   0%|          | 0/3765 [00:00<?, ? examples/s]

KeyError: 'text'

In [65]:
import wandb, os
wandb.login()
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="amh_lama",
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


**Fine-tuning process using Single GPU**

In [66]:
def train(model, tokenizer, dataset, output_dir):
    model.gradient_checkpointing_enable()

    model = prepare_model_for_kbit_training(model)
    modules = find_all_linear_names(model)

    peft_config = create_peft_config(modules)
    model = get_peft_model(model, peft_config)

    print_trainable_parameters(model)

    trainer = Trainer(
        model=model,
        train_dataset=dataset,
        eval_dataset=dataset_subset,
        args=TrainingArguments(
            per_device_train_batch_size=1,
            gradient_accumulation_steps=10,
            warmup_steps=10,
            max_steps=20,
            learning_rate=2.5e-5,
            fp16=True,
            logging_steps=2,
            eval_steps=10,
            do_eval=True,
            output_dir="outputs",
            optim="paged_adamw_8bit",
            report_to="wandb"
        ),
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
    )

    model.config.use_cache = False  


    dtypes = {}
    for _, p in model.named_parameters():
        dtype = p.dtype
        if dtype not in dtypes: dtypes[dtype] = 0
        dtypes[dtype] += p.numel()
    total = 0
    for k, v in dtypes.items(): total+= v
    for k, v in dtypes.items():
        print(k, v, v/total)

    do_train = True

    # Launch training
    print("Training...")

    if do_train:
        train_result = trainer.train()
        metrics = train_result.metrics
        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()
        print(metrics)

    ###

    # Saving model
    print("Saving last checkpoint of the model...")
    os.makedirs(output_dir, exist_ok=True)
    trainer.model.save_pretrained(output_dir)

    # Free memory for merging weights
    del model
    del trainer
    torch.cuda.empty_cache()
namespace = "results"
repo_name = "stableai"
output_dir = f"{namespace}/{repo_name}"
train(model, tokenizer2, dataset, output_dir)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


all params: 1,551,700,992 || trainable params: 25,034,752 || trainable%: 1.6133747499724482
torch.float32 282924032 0.1823315403280995
torch.uint8 1268776960 0.8176684596719005
Training...


/opt/miniconda/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
../aten/src/ATen/native/cuda/Indexing.cu:1290: indexSelectLargeIndex: block: [478,0,0], thread: [64,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1290: indexSelectLargeIndex: block: [478,0,0], thread: [65,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1290: indexSelectLargeIndex: block: [478,0,0], thread: [66,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1290: indexSelectLargeIndex: block: [478,

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

**Merge weights**

In [69]:

model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)
model = model.merge_and_unload()

In [70]:
output_merged_dir = "results/stableai/final_merged_checkpoint"
os.makedirs(output_merged_dir, exist_ok=True)


In [71]:
# save tokenizer for easy inference
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.save_pretrained(output_merged_dir)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('results/stableai/final_merged_checkpoint/tokenizer_config.json',
 'results/stableai/final_merged_checkpoint/special_tokens_map.json',
 'results/stableai/final_merged_checkpoint/tokenizer.json')

In [72]:
#model.save_pretrained(output_merged_dir, safe_serialization=True)


In [67]:
def create_prompt_formats_for_test(sample):
    """
    Formats a sample dictionary into a prompt suitable for fine-tuning Mistral for ad classification.

    Args:
        sample: A dictionary containing 'text' and 'label' fields.

    Returns:
        The modified sample dictionary with the formatted prompt.
    """
    INTRO_BLURB = "You are tasked with classifying messages as either advertisements (Ad) or non-advertisements (Not Ad) based on a comprehensive analysis of various factors."

    INSTRUCTION_KEY = "### Text:"

    blurb = f"{INTRO_BLURB}\n\n"
    text = f"{INSTRUCTION_KEY}\n{sample['text']}\n\n"

    formatted_prompt = blurb + text
    sample["text"] = formatted_prompt

    return sample


In [68]:
prompts = []
for prompt in dataset_subset:
    prompts.append(create_prompt_formats_for_test(prompt)['text'])

In [69]:
import time

**Inference using Instruction or Question Only**


In [70]:
input_texts = []
for prompt in prompts:
    input_texts.append(f"Instruction: {prompt}")

In [71]:
ad = input_texts[83]

In [72]:
not_ad = input_texts[777]
not_ad

'Instruction: You are tasked with classifying messages as either advertisements (Ad) or non-advertisements (Not Ad) based on a comprehensive analysis of various factors.\n\n### Text:\nቻይና የሚገኙ ተማሪዎችን ወደ ኢትዮጵያ ለማምጣት ግብረኃይል ተቋቋመ!  በኮሮና ቫይረስ ስጋት ውስጥ እንደሆኑ የተነገረላቸው ተማሪዎችን ለማስመጣት ግብረኃይል መቋቋሙን የውጭ ጉዳይ ሚኒስቴር ዛሬ አርብ የካቲት 6/2012 በወቅታዊ ጉዳይ በሰጠው ጋዜጣዊ መግለጫ ላይ አስታወቀ፡፡ በቻይና ለሚገኙ ኢትዮጵያውያን ተማሪዎች መንግስት የገንዘብ ድጋፍ መላኩን የገለጹት የሚኒስቴሩ ቃል አቀባይ ነብያት ጌታቸው ተማሪዎቹ በቫይረስ እንዳይጠቁ ክትትል እየተደረገ ነው ብለዋል።  Via Addis Maleda @YeneTube @FikerAssefa\n\n'

In [73]:
def format_input(sample):
    """
    Formats a sample dictionary into a prompt suitable for fine-tuning Mistral for ad classification.

    Args:
        sample: A dictionary containing 'text' and 'label' fields.

    Returns:
        The modified sample dictionary with the formatted prompt.
    """
    INTRO_BLURB = "You are tasked with classifying messages as either advertisements (Ad) or non-advertisements (Not Ad) based on a comprehensive analysis of various factors."

    INSTRUCTION_KEY = "### Text:"

    blurb = f"{INTRO_BLURB}\n\n"
    text = f"{INSTRUCTION_KEY}\n{sample}\n\n"

    formatted_prompt = blurb + text

    return formatted_prompt


In [74]:
input_ids = tokenizer.encode(not_ad, return_tensors="pt", truncation=True).to(model.device)

start_time = time.time()

output = model.generate(input_ids, max_length=max_length, temperature=1.0, top_k=50, top_p=0.95, num_return_sequences=1)
generated_output = tokenizer.decode(output[0], skip_special_tokens=True)

end_time = time.time()

# Calculate and print the inference time
inference_time = end_time - start_time


NameError: name 'tokenizer' is not defined

In [81]:
# Print the formatted input
print(f"TEST \n")
print(f"Generated Output:\n======================\n{generated_output}\n")
print(f"Inference Time:{inference_time} seconds\n==========================================")

TEST 

Generated Output:
Instruction: You are tasked with classifying messages as either advertisements (Ad) or non-advertisements (Not Ad) based on a comprehensive analysis of various factors.

### Text:
ቻይና የሚገኙ ተማሪዎችን ወደ ኢትዮጵያ ለማምጣት ግብረኃይል ተቋቋመ!  በኮሮና ቫይረስ ስጋት ውስጥ እንደሆኑ የተነገረላቸው ተማሪዎችን ለማስመጣት ግብረኃይል መቋቋሙን የውጭ ጉዳይ ሚኒስቴር ዛሬ አርብ የካቲት 6/2012 በወቅታዊ ጉዳይ በሰጠው ጋዜጣዊ መግለጫ ላይ አስታወቀ፡፡ በቻይና ለሚገኙ ኢትዮጵያውያን ተማሪዎች መንግስት የገንዘብ ድጋፍ መላኩን የገለጹት የሚኒስቴሩ ቃል አቀባይ ነብያት ጌታቸው ተማሪዎቹ በቫይረስ እንዳይጠቁ ክትትል እየተደረገ ነው ብለዋል።  Via Addis Maleda @YeneTube @FikerAssefa


### Explanation:
The message is an advertisement for a product called "ሚገኙ ተማሪዎችን ወደ ኢትዮጵያ ለማምጣት ግብረኃይል ተቋቋመ!" which translates to "Get rid of bad credit with ease!" The message also mentions that the product has helped many people to improve their credit score. The message ends with a call to action to buy the product through a specific website. This message is an advertisement for a product that promises to help people improve their credit score. The mess